In [1]:
import numpy as np
import gudhi
import mdtraj as md
import seaborn as sns
import itertools
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
traj = md.load_dcd('deca56D_prd.dcd', top='deca56D_clean.pdb')
traj = md.Trajectory.superpose(traj, traj[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

traj3 = md.load('bottleneck_selected_trajectory.pdb')
traj3 = md.Trajectory.superpose(traj3, traj3[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

rmsd = md.rmsd(traj3, traj3[0], frame=0, atom_indices=None, parallel=True, precentered=False)*10

np.save('BottleneckRMSD.npy',rmsd)

In [ ]:
from pylab import *
from math import pi
time_steps = np.arange(0, 200)

fig, ax = plt.subplots(figsize=(12,6))
inty_ax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0,
                            0.15, ax.get_position().height])

ax.plot(time_steps, rmsd, marker='o', markersize=0, linestyle='-', lw=3)
ax.set_xlabel('Time Steps [ps]', fontsize = 15)
ax.set_ylabel('RMSD Value [Å]', fontsize = 15)
ax.tick_params(labelsize = 15)
sns.kdeplot(y = rmsd, ax = inty_ax, lw=3)
inty_ax.set_xlabel('Density', fontsize=15)
inty_ax.tick_params(labelleft = False, labelsize = 15)

fig.suptitle('56D-decapeptide RMSD', fontsize = 20)
plot_filename = f'selected frame RMSD.png'
plt.savefig(plot_filename, bbox_inches='tight', dpi=900)
plt.show()

subset_traj = traj[1642600:1642700]
subset_traj.save('bsheetbottleframes.pdb')

subset_traj = traj[100100:100200]
subset_traj.save('bsheetbottleframes.pdb')

num_frames = min(traj.n_frames, 1400000)

selected_frames = range(0, num_frames, 7000)
selected_traj = traj[selected_frames]

# Save the selected trajectories into a new PDB file
selected_traj.save_pdb('bottleneck_selected_trajectory.pdb')

In [3]:
atomic_coordinates = traj.xyz

In [4]:
for frame in range(traj.n_frames):
    atomic_coordinates = traj.xyz[frame]

In [5]:
frame2 = traj.xyz[0]

In [32]:
carbon_atoms = traj.topology.select("element == 'C'")

In [33]:
frame1 = traj.xyz[0, carbon_atoms, :]

In [8]:
num_rows = np.size(frame1, axis=0)
print(f"Number of rows: {num_rows}")

Number of rows: 60


In [9]:
num_rows = np.size(frame2, axis=0)
print(f"Number of rows: {num_rows}")

Number of rows: 193


In [34]:
frame3 = traj.xyz[1642600, carbon_atoms, :]
frame4 = traj.xyz[1642600]

In [11]:
rips_complex = gudhi.RipsComplex(points=frame1)

In [12]:
simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)

In [13]:
persistence = simplex_tree.persistence()

In [20]:
alpha_complex = {tuple(sorted(simplex)) : radius for simplex, radius in simplex_tree.get_filtration()}
boundary_matrix = {simplex : set(itertools.combinations(simplex, len(simplex)-1))-{tuple()} for simplex in alpha_complex}

R = { k : v for k,v in boundary_matrix.items()}
V = { k : {k} for k in boundary_matrix}
lowinv = {} # lowinv[i]=index of column with the lowest 1 at i
order_function = lambda s: (alpha_complex[s], s)
for s in sorted(R, key=order_function):
    t = lowinv.get(max(R[s], key=order_function),-1) if len(R[s])!=0 else -1
    while t!=-1:
        R[s] = R[t]^R[s] # symmetric difference of t-th and s-th columns
        V[s] = V[t]^V[s]
        t = lowinv.get(max(R[s], key=order_function),-1) if len(R[s])!=0 else -1
    if len(R[s])!=0:
        lowinv[max(R[s], key=order_function)] = s

loops = [bar for bar in lowinv.items() if len(bar[0])==2]
longest_loop = max(loops, key=lambda bar: alpha_complex[bar[1]]-alpha_complex[bar[0]])
representative_1 = R[longest_loop[1]]
representative_2 = V[longest_loop[0]]
#for edge in representative_1:
    #plt.plot( *zip(frame1[edge[0]], frame1[edge[1]]), '-', color='blue', alpha=.6 )
#for edge in representative_2:
 #   plt.plot( *zip(frame1[edge[0]], frame1[edge[1]]), '--', color='red', alpha=.6 )
#plt.plot(*zip(*frame1),'o')
#plt.show()

In [19]:
print(representative_1)
print(representative_2)

{(26, 27), (25, 30), (42, 46), (36, 37), (37, 42), (27, 46), (30, 36), (25, 26)}
{(37, 42), (45, 46), (27, 46), (44, 45), (26, 27), (24, 26), (31, 36), (42, 44), (24, 25), (30, 31), (25, 30), (36, 37)}


In [42]:
carbon_atoms[37]

120

import matplotlib.pyplot as plt
gudhi.plot_persistence_diagram(persistence)


gudhi.plot_persistence_barcode(persistence)

persistence

In [31]:
# Specify the birth and death times of interest
birth_time = 0.38901214076285695
death_time = 0.5037974522762462

# Find simplices contributing to the birth
birth_simplices = [simplex for simplex, time in simplex_tree.get_simplices() if time == birth_time]

# Find simplices contributing to the death
death_simplices = [simplex for simplex, time in simplex_tree.get_simplices() if time == death_time]
print("Simplices contributing to birth at time", birth_time, ":", birth_simplices)
print("Simplices contributing to death at time", death_time, ":", death_simplices)

Simplices contributing to birth at time 0.38901214076285695 : [[27, 46]]
Simplices contributing to death at time 0.5037974522762462 : [[25, 26, 27, 46], [25, 26, 28, 46], [25, 26, 46], [25, 27, 28, 46], [25, 27, 46], [25, 28, 46], [25, 36, 46], [25, 46]]


first_frame = traj[0]
first_frame.save("first_frame_output.pdb")

In [ ]:
b_sheet = traj[1642600]
b_sheet.save("bsheet_frame_output.pdb")

In [3]:
frame = traj[0:100]
frame.save("normal100frame.pdb")

In [10]:
a_helix = traj[1642550:1642650]
a_helix.save("ahelix100frame.pdb")

In [5]:
traj5 = md.load('normal100frame.pdb')
traj5 = md.Trajectory.superpose(traj5, traj5[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

In [11]:
traj6 = md.load('ahelix100frame.pdb')
traj6 = md.Trajectory.superpose(traj6, traj6[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)